In [5]:
import pandas as pd
import cufflinks as cf

# Настройка режима офлайн для работы с cufflinks и plotly
cf.go_offline()

# Загрузка данных из файла
data = pd.read_csv('Sunspots.csv', low_memory=False, infer_datetime_format=True, index_col=['Date'], dayfirst=True)
data = data.rename(columns={'Monthly Mean Total Sunspot Number':'Sunspots'})
data = data.drop('Unnamed: 0', axis=1)
# Визуализация данных (например, потребление энергии по глобальной активной мощности)
data['Sunspots'].iplot(title='Sunspots', xTitle='Date', yTitle='Sunspots')


/tmp/ipykernel_250587/65387575.py:8: FutureWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [6]:
data

,Sunspots
Date,
1749-01-31,96.7
1749-02-28,104.3
1749-03-31,116.7
1749-04-30,92.8
1749-05-31,141.7
...,...
2020-09-30,0.6
2020-10-31,14.4
2020-11-30,34.0


In [7]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Sunspots']])


In [10]:
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 10  # Например, используем последние 10 значений для прогнозирования следующего
x, y = create_dataset(scaled_data, look_back)




In [11]:
X_dif = x[:-1] - x[1:]

In [12]:
X_dif.shape

(3254, 10)

In [13]:
pd.DataFrame(x)

,0,1,2,3,4,5,6,7,8,9
0,0.242843,0.261929,0.293069,0.233049,0.355851,0.349573,0.396786,0.277499,0.317680,0.315922
1,0.261929,0.293069,0.233049,0.355851,0.349573,0.396786,0.277499,0.317680,0.315922,0.663737
2,0.293069,0.233049,0.355851,0.349573,0.396786,0.277499,0.317680,0.315922,0.663737,0.356605
3,0.233049,0.355851,0.349573,0.396786,0.277499,0.317680,0.315922,0.663737,0.356605,0.306881
4,0.355851,0.349573,0.396786,0.277499,0.317680,0.315922,0.663737,0.356605,0.306881,0.317680
...,...,...,...,...,...,...,...,...,...,...
3250,0.001256,0.003767,0.015570,0.000502,0.003767,0.013059,0.000502,0.014566,0.015319,0.018835
3251,0.003767,0.015570,0.000502,0.003767,0.013059,0.000502,0.014566,0.015319,0.018835,0.001507
3252,0.015570,0.000502,0.003767,0.013059,0.000502,0.014566,0.015319,0.018835,0.001507,0.036163
3253,0.000502,0.003767,0.013059,0.000502,0.014566,0.015319,0.018835,0.001507,0.036163,0.085384


In [14]:
X_new = np.hstack([x[:-1],X_dif])
X_new.shape

(3254, 20)

In [15]:
y = y[:-1]

In [16]:
# Преобразование в форму, которую ожидает RNN
x = np.reshape(X_new, (X_new.shape[0], X_new.shape[1], 1))

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense


In [19]:
model = Sequential()

# Входной слой
model.add(SimpleRNN(50, activation='relu', input_shape=(x_train.shape[1], 1), return_sequences=True))

# Дополнительные слои (если необходимо)
model.add(SimpleRNN(50, activation='relu', return_sequences=True))
model.add(SimpleRNN(50, activation='relu'))

# Выходной слой
model.add(Dense(1))


In [38]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [39]:
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))
# на relu ошибка уменьшается медленнее


Epoch 1/30


82/82 [==============================] - 8s 57ms/step - loss: 0.0159 - val_loss: 0.0056
Epoch 2/30
82/82 [==============================] - 5s 65ms/step - loss: 0.0065 - val_loss: 0.0061
Epoch 3/30
82/82 [==============================] - 6s 78ms/step - loss: 0.0060 - val_loss: 0.0047
Epoch 4/30
82/82 [==============================] - 5s 66ms/step - loss: 0.0056 - val_loss: 0.0040
Epoch 5/30
82/82 [==============================] - 6s 76ms/step - loss: 0.0053 - val_loss: 0.0038
Epoch 6/30
82/82 [==============================] - 5s 63ms/step - loss: 0.0052 - val_loss: 0.0053
Epoch 7/30
82/82 [==============================] - 5s 60ms/step - loss: 0.0048 - val_loss: 0.0036
Epoch 8/30
82/82 [==============================] - 6s 71ms/step - loss: 0.0045 - val_loss: 0.0035
Epoch 9/30
82/82 [==============================] - 5s 59ms/step - loss: 0.0046 - val_loss: 0.0046
Epoch 10/30
82/82 [==============================] - 4s 53ms/step - loss: 0.0041 - val_loss: 0.0036
Epoch 11/30
82/82 [=

In [40]:
predictions = model.predict(x_test)
predicted_power = scaler.inverse_transform(predictions)  # Если вы ранее использовали нормализацию, то выполните обратное преобразование.

# Затем вы можете сравнить "predicted_power" с вашими исходными данными и оценить точность прогноза.


21/21 [==============================] - 1s 16ms/step


In [41]:
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

In [42]:
from sklearn.metrics import mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)

print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: inf%
MAE: 1.6944956907295776


/tmp/ipykernel_250587/3619137022.py:4: RuntimeWarning:

divide by zero encountered in divide



SimpleRNN в Keras является одним из слоев для рекуррентной обработки, и у него есть множество атрибутов и параметров, которые можно настроить. Ниже приведены основные атрибуты и их описания:

units: Положительное целое число, определяющее размерность выходного пространства (т.е. количество нейронов в слое).

activation: Функция активации для использования в ячейке. По умолчанию 'tanh'.

use_bias: Логическое значение, указывающее, следует ли слою использовать смещение.

kernel_initializer: Инициализатор для весов матрицы ядра.

recurrent_initializer: Инициализатор для весов рекуррентной матрицы.

bias_initializer: Инициализатор смещения.

kernel_regularizer: Функция регуляризации, применяемая к матрице ядра.

recurrent_regularizer: Функция регуляризации, применяемая к рекуррентной матрице.

bias_regularizer: Функция регуляризации, применяемая к вектору смещения.

activity_regularizer: Функция регуляризации, применяемая к выходу слоя.

kernel_constraint: Ограничение, применяемое к матрице ядра.

recurrent_constraint: Ограничение, применяемое к рекуррентной матрице.

bias_constraint: Ограничение, применяемое к вектору смещения.

dropout: Доля входных единиц для отсева на каждом обновлении во время обучения.

recurrent_dropout: Доля рекуррентных единиц для отсева на каждом обновлении во время обучения.

return_sequences: Логическое значение. Если истина, возвращает полную последовательность к последнему слою для каждого входа. Если ложь, возвращает только последний выход.

return_state: Логическое значение. Если истина, кроме выходных значений, возвращает также последние состояния.

go_backwards: Логическое значение. Если истина, обрабатывает входные последовательности в обратном порядке.

stateful: Логическое значение. Если истина, слой будет рассматриваться как состояние, которое может быть передано в следующий слой.

unroll: Логическое значение. Если истина, сеть будет раскрывать рекуррентные циклы, что может ускорить процесс, но при этом будет использовать больше памяти.

input_dim: Размерность входных данных. Обычно не требуется, если первым слоем модели является слой Embedding или другой слой, который определяет input_shape.

input_length: Длина входных последовательностей. Обычно не требуется, если первым слоем модели является слой Embedding.

In [25]:
from tensorflow.keras.layers import LSTM, Dense

In [43]:
model = Sequential()

# Входной слой LSTM
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))

# Дополнительные слои (если необходимо)
model.add(LSTM(50, activation='tanh', return_sequences=True))
model.add(LSTM(50, activation='tanh'))

# Выходной слой
model.add(Dense(1))


In [44]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [45]:
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/30
82/82 [==============================] - 10s 81ms/step - loss: 0.0130 - val_loss: 0.0053
Epoch 2/30
82/82 [==============================] - 5s 66ms/step - loss: 0.0058 - val_loss: 0.0042
Epoch 3/30
82/82 [==============================] - 7s 81ms/step - loss: 0.0055 - val_loss: 0.0038
Epoch 4/30
82/82 [==============================] - 6s 70ms/step - loss: 0.0048 - val_loss: 0.0034
Epoch 5/30
82/82 [==============================] - 6s 67ms/step - loss: 0.0043 - val_loss: 0.0028
Epoch 6/30
82/82 [==============================] - 6s 75ms/step - loss: 0.0033 - val_loss: 0.0018
Epoch 7/30
82/82 [==============================] - 7s 85ms/step - loss: 0.0013 - val_loss: 2.1147e-04
Epoch 8/30
82/82 [==============================] - 7s 80ms/step - loss: 2.1495e-04 - val_loss: 8.5672e-05
Epoch 9/30
82/82 [==============================] - 7s 81ms/step - loss: 1.0887e-04 - val_loss: 7.0506e-05
Epoch 10/30
82/82 [==============================] - 5s 61ms/step - loss: 8.1483e-05 - v

In [46]:
predictions = model.predict(x_test)
predicted_power = scaler.inverse_transform(predictions)  # Если вы ранее использовали нормализацию, то выполните обратное преобразование.

# Затем вы можете сравнить "predicted_power" с вашими исходными данными и оценить точность прогноза.


21/21 [==============================] - 1s 20ms/step


In [47]:
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

В TensorFlow (с использованием Keras API), слой LSTM имеет множество атрибутов и параметров. Ниже приведены некоторые из них:

units: Количество LSTM ячеек (или скрытых состояний) в слое.

activation: Функция активации, используемая для обновления скрытых состояний. По умолчанию это "tanh".

recurrent_activation: Функция активации, используемая для забывания, входного и выходного вентилей. По умолчанию это "sigmoid".

use_bias: Булево значение, указывающее, следует ли использовать смещение в слое.

kernel_initializer, recurrent_initializer, bias_initializer: Инициализаторы, которые определяют стратегию инициализации весов.

dropout, recurrent_dropout: Процент отсева для входных данных и рекуррентных данных соответственно.

return_sequences: Булево значение. Если True, возвращает весь последовательный выход; если False, возвращает только последний выход.

return_state: Булево значение. Если True, возвращает состояние вдобавок к выходу.

go_backwards: Булево значение. Если True, обрабатывает входные последовательности в обратном порядке.

stateful: Булево значение. Если True, слой будет использовать состояния во время обучения и прогнозирования.

unroll: Булево значение. Если True, сеть будет развертывать рекурсию, что может ускорить процесс обучения, но потребует больше памяти.

implementation: Один из {1, 2}. Рекомендуется использовать 2, так как это более эффективная реализация, особенно для GPU.

In [48]:
from sklearn.metrics import mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)

print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: inf%
MAE: 1.9036490008063305


/tmp/ipykernel_250587/3619137022.py:4: RuntimeWarning:

divide by zero encountered in divide



In [49]:
from tensorflow.keras.layers import GRU, Dense

In [51]:
model = Sequential()

# Входной слой GRU
model.add(GRU(50, activation='relu', return_sequences=True, input_shape=(x_train.shape[1], 1)))

# Дополнительные слои (если необходимо)
model.add(GRU(50, return_sequences=True))
model.add(GRU(50))

# Выходной слой
model.add(Dense(1))


In [52]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [53]:
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/30


82/82 [==============================] - 12s 93ms/step - loss: 0.0159 - val_loss: 0.0042
Epoch 2/30
82/82 [==============================] - 8s 93ms/step - loss: 0.0052 - val_loss: 0.0042
Epoch 3/30
82/82 [==============================] - 6s 75ms/step - loss: 0.0032 - val_loss: 0.0013
Epoch 4/30
82/82 [==============================] - 8s 92ms/step - loss: 7.7707e-04 - val_loss: 1.7722e-04
Epoch 5/30
82/82 [==============================] - 7s 82ms/step - loss: 2.0074e-04 - val_loss: 6.9355e-05
Epoch 6/30
82/82 [==============================] - 7s 82ms/step - loss: 1.3843e-04 - val_loss: 1.1311e-04
Epoch 7/30
82/82 [==============================] - 6s 76ms/step - loss: 9.8898e-05 - val_loss: 6.2822e-05
Epoch 8/30
82/82 [==============================] - 6s 77ms/step - loss: 7.3840e-05 - val_loss: 3.0314e-05
Epoch 9/30
82/82 [==============================] - 8s 93ms/step - loss: 9.1509e-05 - val_loss: 2.3862e-04
Epoch 10/30
82/82 [==============================] - 6s 74ms/step - los

In [55]:
predictions = model.predict(x_test)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)


21/21 [==============================] - 1s 20ms/step


In [56]:
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

In [57]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)

print(f"MAPE: {mape}%")
print(f"MAE: {mae}")


MAPE: inf%
MAE: 0.575946634370382


/tmp/ipykernel_250587/1740996225.py:3: RuntimeWarning:

divide by zero encountered in divide



In [74]:
np.min(y_test_inv )

0.0

units: Количество GRU ячеек (или скрытых состояний) в слое.

activation: Функция активации для обновления скрытых состояний. По умолчанию "tanh".

recurrent_activation: Функция активации, используемая для обновления и сброса вентилей. По умолчанию "sigmoid".

use_bias: Булево значение, определяющее, следует ли использовать смещение в слое.

kernel_initializer, recurrent_initializer, bias_initializer: Инициализаторы для инициализации различных весовых матриц и смещений.

dropout, recurrent_dropout: Процент отсева для входных данных и рекуррентных данных соответственно.

return_sequences: Булево значение. Если True, возвращает весь последовательный выход; если False, возвращает только последний выход.

return_state: Булево значение. Если True, возвращает состояние вдобавок к выходу.

go_backwards: Булево значение. Если True, обрабатывает входные последовательности в обратном порядке.

stateful: Булево значение. Если True, слой будет сохранять свое состояние между пакетами, что позволяет обрабатывать длинные последовательности.

unroll: Булево значение. Если True, сеть будет развертывать рекурсию, что может ускорить процесс обучения, но потребует больше памяти.

implementation: Один из {1, 2}. Рекомендуется использовать 2, так как это более эффективная реализация, особенно для GPU.

reset_after: Булево значение. Определяет, когда следует применять сброс вентиля (перед или после матричного умножения). Это связано с различием между оригинальной версией GRU и версией, предложенной в "Learning to Execute".

In [103]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class AttentionMechanism(Layer):
    def __init__(self, return_sequences=True):
        super(AttentionMechanism, self).__init__()
        self.return_sequences = return_sequences
        
    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                               initializer="normal")
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                               initializer="zeros")
        
    def call(self, x):
        e = tf.nn.tanh(tf.matmul(x,self.W)+self.b)
        a = tf.nn.softmax(e, axis=1)
        output = x*a
        
        if self.return_sequences:
            return output
        
        return tf.reduce_sum(output, axis=1)


In [112]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(AttentionMechanism(return_sequences=True))  # Внимание после LSTM
model.add(LSTM(50, return_sequences=False))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')


In [113]:
model.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/30


82/82 [==============================] - 8s 55ms/step - loss: 0.0180 - val_loss: 0.0062
Epoch 2/30
82/82 [==============================] - 4s 46ms/step - loss: 0.0072 - val_loss: 0.0060
Epoch 3/30
82/82 [==============================] - 4s 46ms/step - loss: 0.0072 - val_loss: 0.0066
Epoch 4/30
82/82 [==============================] - 4s 44ms/step - loss: 0.0065 - val_loss: 0.0055
Epoch 5/30
82/82 [==============================] - 4s 50ms/step - loss: 0.0060 - val_loss: 0.0052
Epoch 6/30
82/82 [==============================] - 4s 45ms/step - loss: 0.0060 - val_loss: 0.0059
Epoch 7/30
82/82 [==============================] - 4s 52ms/step - loss: 0.0055 - val_loss: 0.0041
Epoch 8/30
82/82 [==============================] - 4s 46ms/step - loss: 0.0056 - val_loss: 0.0043
Epoch 9/30
82/82 [==============================] - 4s 48ms/step - loss: 0.0055 - val_loss: 0.0040
Epoch 10/30
82/82 [==============================] - 5s 59ms/step - loss: 0.0053 - val_loss: 0.0040
Epoch 11/30
82/82 [=

In [114]:
predictions = model.predict(x_test)


21/21 [==============================] - 1s 13ms/step


In [115]:
predictions = model.predict(x_test)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)

21/21 [==============================] - 1s 22ms/step


In [116]:
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
predictions_inv = scaler.inverse_transform(predictions)
df_test = pd.DataFrame({
    'Y_test': y_test_inv.ravel(),
    'Predictions': predictions_inv.ravel()
})
# Отобразить на графике
df_test.iplot(title="Прогноз vs Реальные значения", xTitle="Время", yTitle="Значение", theme="solar")

In [117]:
# def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test_inv, predictions_inv)
mae = mean_absolute_error(y_test_inv, predictions_inv)

print(f"MAPE: {mape}%")
print(f"MAE: {mae}")

MAPE: inf%
MAE: 13.392390220725591


/tmp/ipykernel_250587/1740996225.py:3: RuntimeWarning:

divide by zero encountered in divide

